In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [12]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [13]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
#username = 'i9o5yf5bh59gq1nttz2iic006'
scope = spotify_creds['top_artist_scope']
redirect_uri = spotify_creds['top_artist_redirect_url']

In [ ]:
spotify_creds['saved_library_token']

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [10]:
token

'BQDWJYYXNHAECjcq9XQPWxBjZ7BVOBymuoYcCl2bI-v4lOrgv8FaklI5j7x9tSA0FiBdMz6lN1T754BKkdYq71IqnMb5sNTdqRH6Gv8n-GMsD3VkKFs2yLPR2veNTMrO9fMspTC2rDOvOIFVMoW7O9QZVJWk8ynhH5TbZxWGwSiBdQ'

In [6]:
token = 'BQDs3l4g6Gm5JFXDE4C7ngKQ9Z_7fMFlkN0oL5WwlTT7L63HjMppDNLd_Ml7ZmrwoS6zjBrsEg0-2Wr0qGHB0ACrsd5P2fMvcmayL1vhiu3wXRVXgjR2IkMgB-60EPKOoyXkZCssUb8CwJlC2CYiYPffftUK_-yRtzAxTbZwKauGf1yJ1Kk'

In [27]:
token ='BQBiFkEnJjpqU0h7KL3-6UHUd9H_sQrdefo_U_-Z_943fAmo8eTZyw2XhXHXtUv2_zBVPEMNV9lCfrQqWULYeJc3ekufHJnf7UhUX1WGwqbAale5fa9Q771MNPsqPrtBVsAmRzx_XLJ5aHs12BvWbQaD3i5WOzz_bxzXdrHI9FC4Q35RsLPPRvYtJ6w'

In [28]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [13]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [7]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [9]:
df = get_saved_library()

In [21]:
def get_token(scope):
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    
    return token

In [22]:
get_token('user-library-read')

'BQAxjL1PjczgPk64RIqPXv43SRL7XRqBOP5FQ204A5wekEW2GI6Sncu0V7HoTV6wNiLzWWbqme-QFehtB0Kw5lqHd3FlplPe0XkrCGX3FW1bys1rg7w3rKegGB-dm9LFATI5bcHR82tss4I1Z9IvyRx2jn6o3geMnPlDRkBW-BACy98rQE4'

In [23]:
get_token('user-read-recently-played')

'BQDRwwX-AQFax7ClVapVJNzBb1DVVtsD4cuAmThTXOY8VDBXJxcqyqgvcpPgpkAZp9Sw4dtJ8b3j7WOrSGK0aGU8GHwIB1BIoAaeHnGdgtZealOjqX-xhAlohNc9bsgmP00Q34BAOmJDr3jXHoBHLQFyUjmS76aXH-GoQ8gWBno'

In [38]:
name_c = [get_token('user-read-recently-played')]
with open("out.txt", "w+") as f:
    f.write("data = ")
    json.dump({"name": name_c[0]}, f)

In [39]:
f = open("out.txt", "r")

In [42]:
f.read()[0]

IndexError: string index out of range